#### Dependecies

scipy = 0.17.0
gensim = 1.0.1
pandas = 0.19.2
numpy = 1.11.2

#### I used the following code in Pyspark to generate and save on disk the csv file below

select_noteevents2 = spark.sql("""
SELECT text
FROM noteevents2
""")

select_noteevents2.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3", header=True)

select_noteevents3 = spark.sql("""
SELECT subject_id, hadm_id
FROM noteevents2
""")

select_noteevents3.coalesce(1).write.csv("/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v4", header=True)

#### then I proceed to open a new notebook to run pandas and gensim 

In [2]:
import pandas as pd
df = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3/part-00000-ac3edd68-1571-4e16-a2c6-ada909da0eea.csv',sep='delimiter')
df.head(5)

/Users/cesarosorio/anaconda/envs/BDFH_project/lib/python2.7/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,text
0,"""Admission Date: [**2183-9-25**] Discha..."
1,"""Admission Date: [**2184-1-16**] Discha..."
2,"""Admission Date: [**2103-4-11**] ..."
3,"""Admission Date: [**2103-10-7**] Discha..."
4,"""\""Admission Date: [**2131-4-2**] ..."


In [2]:
df_ids = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v4/part-00000-9b6dc461-8bf9-4ae4-b626-2156a12031ab.csv')
df_ids.head(5)

,subject_id,hadm_id
0,20007,188442
1,20007,193793
2,59883,118446
3,17043,157985
4,7019,189488


### Apply Word2Vec

In [26]:
from gensim.models import Word2Vec
#import gensim.models.Word2Vec
from gensim import utils
from time import time

# assumptions: window is 5 words left and right, eliminate words than dont occur in
# more than 10 docs, use 4 workers for a quadcore machine. Size is the size of vector
# negative=5 implies negative sampling and makes doc2vec faster to train
#model = Doc2Vec(sentence, size=100, window=5, workers=4, min_count=5)


import random

size = 100

#instantiate our  model
model_w2v = Word2Vec(min_count=10, window=5, size=size, sample=1e-3, negative=5, workers=4)

#build vocab over all reviews
model_w2v.build_vocab(token_review)

#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
Idx=list(range(len(token_review)))

t0 = time()
for epoch in range(5):
     random.shuffle(Idx)
     perm_sentences = [token_review[i] for i in Idx]
     model_w2v.train(perm_sentences)
     print(epoch)
    
elapsed=time() - t0
print("Time taken for Word2vec training: ", elapsed, "seconds.")

0
1
2
3
4
('Time taken for Word2vec training: ', 766.5733489990234, 'seconds.')


In [1]:
# saves the word2vec model to be used later.
#model_w2v.save('./model_word2vec')

# open a saved word2vec model 
import gensim
model_w2v=gensim.models.Word2Vec.load('./model_word2vec')

In [59]:
#gensim.parsing.preprocessing.remove_stopwords(df.iloc[0]['text2'])
#gensim.parsing.preprocessing.remove_stopwords(token_review[0])

In [2]:
#import data
import pandas as pd
df = pd.read_csv('/Volumes/EXTERNAL1/MIMICIII/select_noteevents2-v3/part-00000-ac3edd68-1571-4e16-a2c6-ada909da0eea.csv',sep='delimiter')

# Cleanning the data
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    text = re.sub(" \d+", " ", text)
    text = gensim.parsing.preprocessing.remove_stopwords(text)
    return text

df['text3'] = df['text'].apply(preprocessor)

# Create tokens
token_review=[]
for i in range(df['text3'].shape[0]):
    review = df['text3'][i]
    token_review.append([i for i in review.split()])

len(token_review)

/Users/cesarosorio/anaconda/envs/BDFH_project/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


59652

In [3]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [4]:
makeFeatureVec(token_review[0], model_w2v, 100)

array([ 0.24209931,  0.57821953, -0.1097922 , -0.04088522,  0.76877069,
        1.2996639 ,  0.41623062,  0.1442489 ,  0.77406311, -0.52265239,
        0.76842451,  0.15562041, -0.46637994,  0.01335333, -0.25037864,
       -0.69962567,  0.49107403,  0.38508704, -0.21999806,  0.18665832,
       -0.13348928, -0.7613135 ,  0.46283022,  0.73189414, -0.2284956 ,
        0.11714122, -0.04166868, -0.77739626,  0.08127863, -0.08135029,
        0.57344264,  0.13657698,  0.02911675, -0.40416399,  0.75751036,
        0.34503514,  0.33877051, -0.43589294,  0.31269494, -0.40871796,
       -0.0605538 , -0.3819671 ,  0.2187874 , -0.12320303,  0.40263268,
        0.56866163,  0.01980313,  0.33025518,  0.75881159,  0.17011712,
       -0.40949446, -0.95047182, -0.07816353,  0.39292824, -1.02860975,
        0.5054608 , -0.16936162, -0.43043894, -0.49423882,  0.09224224,
       -0.44405714, -0.43945932, -0.55873865,  0.37785959, -0.46316189,
        0.03862812, -0.63376772, -0.20108515,  1.03987992, -0.25

In [5]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%10000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [6]:
final_w2v=getAvgFeatureVecs(token_review, model_w2v, num_features=100)

Review 0 of 59652
Review 10000 of 59652
Review 20000 of 59652
Review 30000 of 59652
Review 40000 of 59652
Review 50000 of 59652


In [7]:
# Create train set and test set to use Machine Learning model

import numpy as np
n_break=50000  # numbers of docs in training set
size = 100 # define when running doc2vec
n_final = 59652
X_train_w2v = np.zeros((n_break, size))
X_test_w2v = np.zeros((n_final - n_break, size))

for i in range(len(X_train_w2v)):
    X_train_w2v[i] = final_w2v[i]
    
for i in range(n_final - n_break):
    X_test_w2v[i] = final_w2v[i+n_break]

print(X_train_w2v.shape ,X_test_w2v.shape )

((50000, 100), (9652, 100))


In [8]:
model_w2v.most_similar('melanoma')

[('sarcoma', 0.7263417840003967),
 ('rcc', 0.7242584228515625),
 ('melenoma', 0.7210500240325928),
 ('leiomyosarcoma', 0.7176036834716797),
 ('nsclc', 0.6890457272529602),
 ('fibrosarcoma', 0.6835242509841919),
 ('mesothelioma', 0.6819085478782654),
 ('adenocarcinoma', 0.6768175363540649),
 ('thymoma', 0.6764349937438965),
 ('sclc', 0.6748671531677246)]